# Some Common TensorFlow errors

This notebook demonstrates some common TensorFlow errors, how to find them, and how to fix them.

In [1]:
import tensorflow as tf
print(tf.__version__)

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)


1.8.0


## Shape error

If you run following code, you will get error message:

**ValueError: Dimensions must be equal, but are 2 and 4 for 'add' (op: 'Add') with input shapes: [4,2], [4].**

In [9]:
def some_method(data):
    a = data[:,0:2]
    c = data[:,1]
    s = (a + c)
    return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
    fake_data = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
      [2.8, 4.2, 5.6],
      [2.9, 8.3, 7.3]
    ])
    print(sess.run(some_method(fake_data)))

This error message means that we add two tensors with different shapes. You can use **get_shape()** function or **shape** property of a tensor to get the shape information. 

In [22]:
def some_method(data):
    a = data[:,0:2]
    print("a shape:", a.get_shape(), a.shape)
    c = data[:,1]
    print("c shape:", c.get_shape(), c.shape)
    s = (a + c)
    return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
    fake_data = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
      [2.8, 4.2, 5.6],
      [2.9, 8.3, 7.3]
    ])
    print(sess.run(some_method(fake_data)))

a shape: (4, 2) (4, 2)
c shape: (4,) (4,)


ValueError: Dimensions must be equal, but are 2 and 4 for 'add_9' (op: 'Add') with input shapes: [4,2], [4].

If you run above code, you would get following message:

a shape: (4, 2) 

c shape: (4,)

some error message here ...

The way to fix this error is to basically make sure that both of these tensors are the same shape

In [18]:
def some_method(data):
    a = data[:,0:2]
    print(a.get_shape())
    c = data[:,1:3]  # <------ Note here
    print(c.get_shape())
    s = (a + c)
    return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
    fake_data = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
      [2.8, 4.2, 5.6],
      [2.9, 8.3, 7.3]
    ])
    print(sess.run(some_method(fake_data)))

(4, 2)
(4, 2)
[[12.884487 11.878131 12.449096 15.721323]
 [11.878131 10.962208 11.489996 14.509306]
 [12.449096 11.489996 12.043255 15.207892]
 [15.721323 14.509306 15.207892 19.204166]]


In [5]:
import tensorflow as tf

x = tf.constant([[3, 2],
                 [4, 5],
                 [6, 7]])
print("x.shape", x.shape)
expanded = tf.expand_dims(x, 1)
print("expanded.shape", expanded.shape)
sliced = tf.slice(x, [0, 1], [2, 1])
print("sliced.shape", sliced.shape)

with tf.Session() as sess:
    print("expanded: ", expanded.eval())
    print("sliced: ", sliced.eval())

x.shape (3, 2)
expanded.shape (3, 1, 2)
sliced.shape (2, 1)
expanded:  [[[3 2]]

 [[4 5]]

 [[6 7]]]
sliced:  [[2]
 [5]]


In [24]:
# from tensorflow.python import debug as tf_debug

# def some_method(a, b):
#   b = tf.cast(b, tf.float32)
#   s = (a / b)
#   s2 = tf.matmul(s, tf.transpose(s))
#   return tf.sqrt(s2)

# with tf.Session() as sess:
#   fake_a = [
#       [5.0, 3.0, 7.1],
#       [2.3, 4.1, 4.8],
#     ]
#   fake_b = [
#       [2, 0, 5],
#       [2, 8, 7]
#     ]
#   a = tf.placeholder(tf.float32, shape=[2, 3])
#   b = tf.placeholder(tf.int32, shape=[2, 3])
#   k = some_method(a, b)
  
#   # Note: won't work without the ui_type="readline" argument because
#   # Datalab is not an interactive terminal and doesn't support the default "curses" ui_type.
#   # If you are running this a standalone program, omit the ui_type parameter and add --debug
#   # when invoking the TensorFlow program
#   #      --debug (e.g: python debugger.py --debug )
#   sess = tf_debug.LocalCLIDebugWrapperSession(sess, ui_type="readline")
#   sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)
#   print(sess.run(k, feed_dict = {a: fake_a, b: fake_b}))

In [20]:
%%writefile debugger.py
import tensorflow as tf

def some_method(a, b):
    b = tf.cast(b, tf.float32)
    s = (a / b)
    print_ab = tf.Print(s, [a, b])
    s = tf.where(tf.is_nan(s), print_ab, s)
    return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
    fake_a = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
    ])
    fake_b = tf.constant([
      [2, 0, 5],
      [2, 8, 7]
    ])
  
    print(sess.run(some_method(fake_a, fake_b)))

Writing debugger.py


In [23]:
%%bash
python debugger.py

[[      inf       inf]
 [      inf 1.4336528]]


2019-03-10 23:51:38.778709: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
[[5 3 7.1]...][[2 0 5]...]
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
